In [74]:
import numpy as np
import pandas as pd
from random import random
from random import seed

In [75]:
# initialize a network
# a network is represented as a list of dictionaries
def initialize_network(n_inputs, n_hidden, n_outputs):
    network=list()
    hidden_layer=[{'weights':[random() for i in range(n_inputs+1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    outer_layer=[{'weights':[random() for i in range(n_hidden+1)]} for i in range(n_outputs)]
    network.append(outer_layer)
    return network

In [76]:
# Test a network creation
seed(1)
network=initialize_network(2,1,2)
for layer in network:
    print(layer)
    print()

[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}]

[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]



In [77]:
# Forward propagation
# Activation function: 

def linear(weights, inputs):
    linear=weights[-1] # Store the last weight as bias
    for i in range(len(weights)-1):
        linear += weights[i]*inputs[i]
    return linear

def activation(x):
    return 1.0/(1.0+ np.exp(-x))

def forward_propagation(network, row):
    inputs=row
    for layer in network:
        new_inputs=[]
        for neuron in layer:
            linearComb=linear(neuron['weights'], row)
            neuron['output']=activation(linearComb)
            new_inputs.append(neuron['output'])
        inputs=new_inputs
    return inputs         
            
    

In [78]:
# Test the forward prop
network = [[{'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'weights': [0.2550690257394217, 0.49543508709194095]}, {'weights': [0.4494910647887381, 0.651592972722763]}]]
row=[1,0, None]
output=forward_propagation(network, row)
print(output)

[0.6792885329144388, 0.7504631664645315]


In [79]:
def activation_derivative(x):
    return x * (1.0-x)
def back_propagation(network, expected):
    for i in reversed(range(len(network))):
    
        layer=network[i]
        errors=list()
        # if i is not the last layer do
        if i != len(network)-1:
            for j in range(len(layer)):
                error=0.0
                for neuron in network[i+1]:
                    error+=(neuron['weights'][j] * neuron['delta']) *activation_derivative(layer[j]['output'])
                    errors.append(error)
        else:# if i is the last layer
            for j in range(len(layer)):
                neuron=layer[j]
                errors.append(expected[j]-neuron['output'])
        for j in range(len(layer)):
            
            neuron=layer[j]
            neuron['delta']=errors[j]*activation_derivative(neuron['output'])
        
            
            
        

In [80]:
network = [[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614]}],
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095]}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763]}]]
expected = [0, 1]
back_propagation(network, expected)
for layer in network:
    print(layer)

[{'output': 0.7105668883115941, 'weights': [0.13436424411240122, 0.8474337369372327, 0.763774618976614], 'delta': -0.0015771887491200093}]
[{'output': 0.6213859615555266, 'weights': [0.2550690257394217, 0.49543508709194095], 'delta': -0.14619064683582808}, {'output': 0.6573693455986976, 'weights': [0.4494910647887381, 0.651592972722763], 'delta': 0.0771723774346327}]


In [81]:
def update_weights(network, row, eta):
    for i in range(len(network)):
        inputs=row[:-1]
        if i!=0:
            inputs=[neuron['output'] for neuron in network[i-1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j]+=eta*neuron['delta']*inputs[j]
            neuron['weights'][-1] += eta * neuron['delta']

In [82]:
def train_network(network, train, l_rate, n_epoch, n_outputs):
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            outputs = forward_propagation(network, row)
            expected = [0 for i in range(n_outputs)]
            expected[row[-1]] = 1
            sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
            back_propagation(network, expected)
            update_weights(network, row, l_rate)
        print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))

In [83]:
dataset = [[2.7810836,2.550537003,0],
           [1.465489372,2.362125076,0],
           [3.396561688,4.400293529,0],           
           [1.38807019,1.850220317,0],
           [3.06407232,3.005305973,0],
           [7.627531214,2.759262235,1],
           [5.332441248,2.088626775,1],
           [6.922596716,1.77106367,1],
           [8.675418651,-0.242068655,1],
           [7.673756466,3.508563011,1]]
n_inputs = len(dataset[0]) - 1
n_outputs = len(set([row[-1] for row in dataset]))
network = initialize_network(n_inputs, 2, n_outputs)
train_network(network, dataset, 0.5, 20, n_outputs)
for layer in network:
    print(layer)

>epoch=0, lrate=0.500, error=7.269
>epoch=1, lrate=0.500, error=6.077
>epoch=2, lrate=0.500, error=6.025
>epoch=3, lrate=0.500, error=5.951
>epoch=4, lrate=0.500, error=5.882
>epoch=5, lrate=0.500, error=5.824
>epoch=6, lrate=0.500, error=5.782
>epoch=7, lrate=0.500, error=5.739
>epoch=8, lrate=0.500, error=5.697
>epoch=9, lrate=0.500, error=5.657
>epoch=10, lrate=0.500, error=5.619
>epoch=11, lrate=0.500, error=5.579
>epoch=12, lrate=0.500, error=5.533
>epoch=13, lrate=0.500, error=5.474
>epoch=14, lrate=0.500, error=5.396
>epoch=15, lrate=0.500, error=5.291
>epoch=16, lrate=0.500, error=5.155
>epoch=17, lrate=0.500, error=4.992
>epoch=18, lrate=0.500, error=4.814
>epoch=19, lrate=0.500, error=4.634
[{'weights': [0.7665992372349709, 0.0653135178788911, 0.015889783956363212], 'output': 0.9978232428381898, 'delta': 6.39943721605866e-08}, {'weights': [0.832517249353776, 0.4284676877063701, 0.7604714186299393], 'output': 0.99982530805101, 'delta': 5.6778207567042455e-09}]
[{'weights': [-0